<a href="https://colab.research.google.com/github/Clever-Lazinezz/NoteBook/blob/main/Mini_Project_4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#Principle Component Analysis?? = next topic

## Import Libraries

In [ ]:
!pip install pydot
!pip install pydot-ng
!pip install graphviz
!pip install --upgrade scikit-Learn==0.20.3

In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

## Read Files

In [2]:
df = pd.read_csv('https://raw.githubusercontent.com/Clever-Lazinezz/ML-Data2/main/diabetic_data.csv')

Semi Supervised Code

## EDA

In [3]:
data = df

In [ ]:
data.head()

In [ ]:
data.isnull().count()

### Cleaning up Data

In [4]:
# Chaning age ranges to the median number in the range
# This is done to create integer(or categorical) values that can be used in machine learning models
replaceDict = {'[0-10)' : 5,
'[10-20)' : 15,
'[20-30)' : 25, 
'[30-40)' : 35, 
'[40-50)' : 45, 
'[50-60)' : 55,
'[60-70)' : 65, 
'[70-80)' : 75,
'[80-90)' : 85,
'[90-100)' : 95}

data['age'] = data['age'].apply(lambda x : replaceDict[x])

In [ ]:
data['age'].head()

In [ ]:
data.columns

In [24]:
len(df)

101766

#### Checking for duplicate patients

In [27]:
len(data['encounter_id'].unique())

101766

In [ ]:
len(data['admission_source_id'].unique())

In [28]:
# Looks like there are duplicates
len(data['patient_nbr'].unique())

71518

In [6]:
# dropna removes the rows that contain null values
data.dropna(inplace = True)
data.drop_duplicates(['patient_nbr'], keep = 'first', inplace = True)

In [ ]:
len(data)

#### Dropping Unwanted Data

In [7]:
from collections import Counter

In [ ]:
data.columns

In [ ]:
data.head()

In [ ]:
data.info()

In [ ]:
# Dropping this column due to the insane amount of missing data represented as '?'
data['weight'].value_counts(dropna=False)

In [ ]:
data['encounter_id'].value_counts(dropna=False)

In [ ]:
data['encounter_id'].value_counts(dropna=False)

In [8]:
data.drop(['weight', 'medical_specialty', 'payer_code', 'encounter_id', 'patient_nbr', 'diag_1', 'diag_2', 'diag_3'], axis=1, inplace=True)

In [ ]:
data.columns

#### Replacing remaining objects with integer values

In [ ]:
# DONT REINVENT THE WHEEL - USE YOUR RESOURCES
# Solution by Sachin Ranveer
# Functions could be written in a seperate cell, and then called by name for the apply method
data['discharge_disposition_id'] = data['discharge_disposition_id'].apply(lambda x : 1 if int(x) in [6, 8, 9, 13] 
                                                                           else ( 2 if int(x) in [3, 4, 5, 14, 22, 23, 24]
                                                                           else ( 10 if int(x) in [12, 15, 16, 17]
                                                                           else ( 11 if int(x) in [19, 20, 21]
                                                                           else ( 18 if int(x) in [25, 26] 
                                                                           else int(x) )))))

data = data[~data.discharge_disposition_id.isin([11,13,14,19,20,21])]

data['admission_type_id'] = data['admission_type_id'].apply(lambda x : 1 if int(x) in [2, 7]
                                                            else ( 5 if int(x) in [6, 8]
                                                            else int(x) ))

data['admission_source_id'] = data['admission_source_id'].apply(lambda x : 1 if int(x) in [2, 3]
                                                            else ( 4 if int(x) in [5, 6, 10, 22, 25]
                                                            else ( 9 if int(x) in [15, 17, 20, 21]
                                                            else ( 11 if int(x) in [13, 14]
                                                            else int(x) ))))

for col in ["metformin", "repaglinide", "nateglinide", "chlorpropamide", "glimepiride", "acetohexamide", "glipizide", "glyburide", "tolbutamide", "pioglitazone", "rosiglitazone", "acarbose", "miglitol", "troglitazone", "tolazamide", "examide", "citoglipton", "insulin", "glyburide-metformin", "glipizide-metformin", "glimepiride-pioglitazone", "metformin-rosiglitazone", "metformin-pioglitazone"]:
    data[col] = data[col].apply(lambda x : 10 if x == 'Up' 
                                              else ( -10 if x == 'Down'                                                          
                                              else ( 0 if x == 'Steady'
                                              else  -20)))


data['change'] = data['change'].apply(lambda x : 1 if x == 'Ch'
                                                 else -1)


data['diabetesMed'] = data['diabetesMed'].apply(lambda x : -1 if x == 'No'
                                                else 1)


data['max_glu_serum'] = data['max_glu_serum'].apply(lambda x : 200 if x == '>200' 
                                                            else ( 300 if x == '>300'                                                          
                                                            else ( 100 if x == 'Norm'
                                                            else  0)))

data['A1Cresult'] = data['A1Cresult'].apply(lambda x : 7 if x == '>7' 
                                                         else (8 if  x == '>8'                                                        
                                                         else ( 5 if x == 'Norm'
                                                          else  0)))

In [13]:
data['readmitted'].unique()

array(['NO', '>30', '<30'], dtype=object)

In [ ]:
# Alternate way of doing the following cell's operations
# MISSING STEP - CONVERTING TYPE TO INT(this method still leaves values as objects)
"""
data['readmitted'] = data['readmitted'].replace(['NO'],0)
data['readmitted'] = data['readmitted'].replace(['>30'],45)
data['readmitted'] = data['readmitted'].replace(['<30'],15)
"""

In [ ]:
# Note how parenthesises are placed...
data['readmitted'] = data['readmitted'].apply(lambda x : 0 if x == 'NO'
                                                        else (45 if x == '>30'
                                                        else 15))

In [ ]:
data['readmitted']

In [ ]:
# Solution from Sachin Ranveer
# Coverting diagnosises into usable data will be done on a future date due to the numerous challenges
"""
diag_1 = Counter(list(data['diag_1'])).most_common(1)[0][0]
diag_2 = Counter(list(data['diag_2'])).most_common(1)[0][0]
diag_3 = Counter(list(data['diag_3'])).most_common(1)[0][0]
data['diag_1'] = data['diag_1'].apply(lambda x : diag_1 if x == '?' else x)
data['diag_2'] = data['diag_2'].apply(lambda x : diag_2 if x == '?' else x)
data['diag_3'] = data['diag_3'].apply(lambda x : diag_3 if x == '?' else x)
"""

In [12]:

data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 70434 entries, 0 to 101765
Data columns (total 45 columns):
 #   Column                    Non-Null Count  Dtype 
---  ------                    --------------  ----- 
 0   race                      70434 non-null  object
 1   gender                    70434 non-null  object
 2   age                       70434 non-null  int64 
 3   admission_type_id         70434 non-null  int64 
 4   discharge_disposition_id  70434 non-null  int64 
 5   admission_source_id       70434 non-null  int64 
 6   time_in_hospital          70434 non-null  int64 
 7   num_lab_procedures        70434 non-null  int64 
 8   num_procedures            70434 non-null  int64 
 9   num_medications           70434 non-null  int64 
 10  number_outpatient         70434 non-null  int64 
 11  number_emergency          70434 non-null  int64 
 12  number_inpatient          70434 non-null  int64 
 13  diag_1                    70434 non-null  object
 14  diag_2               

In [ ]:
data.head()

In [15]:
data['race'].value_counts(dropna=False)

Caucasian          52663
AfricanAmerican    12692
?                   1921
Hispanic            1506
Other               1161
Asian                491
Name: race, dtype: int64

## Training Test Split

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
X = df.drop("Kyphosis", axis=1)
y = df["Kyphosis"]

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.33, random_state = 0)

In [ ]:
from sklearn.tree import DecisionTreeClassifier
dtree = DecisionTreeClassifier()
dtree.fit(X_train, y_train)

In [ ]:
predictions = dtree.predict(X_test)
from sklearn.metrics import classification_report, confusion_matrix
print(classification_report(y_test, predictions))

In [ ]:
print(confusion_matrix(y_test, predictions))

[[15  5]
 [ 4  3]]


In [ ]:
from IPython.display import Image
from sklearn.externals.six import StringIO
from sklearn.tree import export_graphviz
import pydot

features = list(df.columns[1:])
features

['Age', 'Number', 'Start']

In [ ]:
dot_data = StringIO()
export_graphviz(dtree, out_file=dot_data, feature_names=features,filled=True, rounded=True)

graph = pydot.graph_from_dot_data(dot_data.getvalue())
Image(graph[0].create_png())

In [ ]:
rfc_pred = rfc.predict(X_test)
print(confusion_matrix(y_test, ref_pred))

In [ ]:
print(classification_report(X))